In [0]:
import os
import pandas as pd

In [0]:
data = None
for file in os.listdir('/content/drive/My Drive/accel_phone_train/'):
    if data is None:
        data = pd.read_csv('/content/drive/My Drive/accel_phone_train/'+file, header=None)
    else:
        data = pd.concat([data, pd.read_csv('/content/drive/My Drive/accel_phone_train/'+file, header=None)], axis=0)

In [3]:
data = data.sort_values([0,1,2])
data=data.reset_index(drop=True)
data.columns = ['id','task','time','r1','r2','r3']
data

,id,task,time,r1,r2,r3
0,1600,A,252207666810782,-0.364761,8.793503,1.0550842;
1,1600,A,252207717164786,-0.879730,9.768784,1.0169983;
2,1600,A,252207767518790,2.001495,11.109070,2.619156;
3,1600,A,252207817872794,0.450623,12.651642,0.18455505;
4,1600,A,252207868226798,-2.164352,13.928436,-4.4224854;
...,...,...,...,...,...,...
1580240,1619,S,173119537417474,-4.827309,-3.670910,7.6470804;
1580241,1619,S,173119576999296,-4.746505,-3.773861,7.619547;
1580242,1619,S,173119616901900,-4.587889,-3.876213,7.666234;
1580243,1619,S,173119656833773,-4.545991,-3.940257,7.635708;


In [0]:
def remove_semicolon(x):
    return float(x[:-1])

In [0]:
data['r3'] = data['r3'].apply(remove_semicolon)

In [6]:
data

,id,task,time,r1,r2,r3
0,1600,A,252207666810782,-0.364761,8.793503,1.055084
1,1600,A,252207717164786,-0.879730,9.768784,1.016998
2,1600,A,252207767518790,2.001495,11.109070,2.619156
3,1600,A,252207817872794,0.450623,12.651642,0.184555
4,1600,A,252207868226798,-2.164352,13.928436,-4.422485
...,...,...,...,...,...,...
1580240,1619,S,173119537417474,-4.827309,-3.670910,7.647080
1580241,1619,S,173119576999296,-4.746505,-3.773861,7.619547
1580242,1619,S,173119616901900,-4.587889,-3.876213,7.666234
1580243,1619,S,173119656833773,-4.545991,-3.940257,7.635708


In [7]:
import keras

Using TensorFlow backend.


In [0]:
def _default_Conv1D(filters, kernel_size):
    return keras.layers.Conv1D(filters=filters,
                              kernel_size=kernel_size,
                              strides=1,
                              activation='relu',
                              use_bias=True,
                              padding='same')
    
def fn(inputs, num_filters):
    z_bottleneck = _default_Conv1D(filters=num_filters, kernel_size=1)(inputs)

    z_maxpool = keras.layers.MaxPool1D(pool_size=3, strides=1, padding='same')(inputs)
    
    z1 = _default_Conv1D(filters=num_filters, kernel_size=10)(z_bottleneck)
    z2 = _default_Conv1D(filters=num_filters, kernel_size=20)(z_bottleneck)
    z3 = _default_Conv1D(filters=num_filters, kernel_size=40)(z_bottleneck)
    z4 = _default_Conv1D(filters=num_filters, kernel_size=1)(z_maxpool)
    
    z = keras.layers.Concatenate(axis=2)([z1,z2,z3,z4])
    z = keras.layers.BatchNormalization()(z)
    
    z = keras.layers.Activation('relu')(z)
    return z


def shortcut_layer(inputs, z_inception):
    z_shortcut = keras.layers.Conv1D(filters=int(z_inception.shape[-1]),kernel_size=1,padding='same',use_bias=False)(inputs)
    z_shortcut = keras.layers.normalization.BatchNormalization()(z_shortcut)
    
    z = keras.layers.Add()([z_shortcut,z_inception])
    return keras.layers.Activation('relu')(z)

def build_model(input_shape, num_classes, num_modules=6):
    input_layer = keras.layers.Input(input_shape)
    z = input_layer
    z_residual = input_layer
    
    for i in range(num_modules):
        z = fn(z, 32)
        if i%3 == 2:
            z = shortcut_layer(z_residual, z)
            z_residual = z
    
    gap_layer = keras.layers.GlobalAveragePooling1D()(z)
    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap_layer)
    
    model = keras.models.Model(inputs = input_layer, outputs = output_layer)
    model.compile(loss = "categorical_crossentropy", optimizer = keras.optimizers.Adam(learning_rate=learning_rate), metrics = ['accuracy'])
    
    return model

In [0]:
import numpy as np

In [10]:
n_labels = len(set(data['task']))
n_labels

18

In [0]:
x = data.groupby(['id', 'task']).apply(lambda x: np.array(x[['r1','r2','r3']].values.tolist()))
users, tasks = [],[]
for user, task in x.index.values:
    users.append(user)
    tasks.append(task)

In [0]:
import random
def sample_ts(x, length):
    sample_index = random.choice(x.index.values)
    task = sample_index[1]
    temp = x[sample_index]
    
    start_index = random.randint(0 , temp.shape[0]-length)
    sample = temp[start_index:start_index+length]
    
    return sample, task

def get_samples(x, length=100, n_samples=2000):
    samples = []
    tasks = []
    for _ in range(n_samples):
        a,b = sample_ts(x, length)
        samples.append(a)
        tasks.append(b)
    return np.array(samples), tasks


In [0]:
length = 200
n_samples=10000
epochs=20
batch_size=16
learning_rate=0.005

In [0]:
new_values, tasks = get_samples(x, length=length, n_samples=n_samples)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()
le_output = le.fit_transform(tasks)

ohe = OneHotEncoder()
ohe_output = ohe.fit_transform(le_output.reshape(-1,1)).toarray()

In [0]:
model = build_model([None, 3], n_labels)

In [18]:
%%time
model.fit(new_values, ohe_output, epochs=epochs, batch_size=batch_size, validation_split=0.0)

Epoch 1/20
10000/10000 [==============================] - 14s 1ms/step - loss: 0.9663 - accuracy: 0.6434
Epoch 2/20
10000/10000 [==============================] - 14s 1ms/step - loss: 0.9321 - accuracy: 0.6542
Epoch 3/20
10000/10000 [==============================] - 14s 1ms/step - loss: 0.9254 - accuracy: 0.6613
Epoch 4/20
10000/10000 [==============================] - 14s 1ms/step - loss: 0.8882 - accuracy: 0.6718
Epoch 5/20
10000/10000 [==============================] - 14s 1ms/step - loss: 0.8437 - accuracy: 0.6921
Epoch 6/20
10000/10000 [==============================] - 15s 1ms/step - loss: 0.8483 - accuracy: 0.6901
Epoch 7/20
10000/10000 [==============================] - 14s 1ms/step - loss: 0.8157 - accuracy: 0.6980
Epoch 8/20
10000/10000 [==============================] - 14s 1ms/step - loss: 0.7779 - accuracy: 0.7127
Epoch 9/20
10000/10000 [==============================] - 14s 1ms/step - loss: 0.7581 - accuracy: 0.7208
Epoch 10/20
10000/10000 [==============================

In [0]:
def load_test_data(dir_ = '/content/drive/My Drive/Algo8/test/phone/accel/'):
    data = None
    for file in os.listdir(dir_):
        if data is None:
            data = pd.read_csv(dir_+file, header=None)
        else:
            data = pd.concat([data, pd.read_csv(dir_+file, header=None)], axis=0)

    data.columns = ['id','task','time','r1','r2','r3']
    data = data.sort_values(['id','task','time'])
    data=data.reset_index(drop=True)

    data['r3'] = data['r3'].apply(remove_semicolon)

    x = data.groupby(['id', 'task']).apply(lambda x: np.array(x[['r1','r2','r3']].values.tolist()))

    return x

In [0]:
test_x = load_test_data('/content/drive/My Drive/Algo8/test/phone/accel/')

In [0]:
test_y = []
for _, task in test_x.index.values:
    test_y.append(task)

In [0]:
def predict_full(model, x):
    predictions = []
    for t in x:
        predictions.append(model.predict(t.reshape([1,t.shape[0], t.shape[1]])))

    labels = []
    for pred in predictions:
        labels.append(ohe.inverse_transform(pred)[0])

    labels = le.inverse_transform(labels)
    return labels

In [24]:
total_preds = predict_full(model, test_x)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
def predict_sample(model, p_x):
    predictions = model.predict(p_x)

    labels = ohe.inverse_transform(predictions)

    labels = le.inverse_transform(labels)
    return labels

In [0]:
p_x, p_y = get_samples(test_x, length=200, n_samples=50000)

In [27]:
sample_preds = predict_sample(model, p_x)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [29]:
### overall
accuracy_score(test_y, total_preds)

0.3055555555555556